**Goal** = calculate the mean and similarities between founders from other datasets

# Collaborative Cross : mouse

http://csbio.unc.edu/CCstatus/CCGenomes/#genotypes

https://www.genetics.org/content/206/2/537

## Prepare data

In [2]:
import numpy as np
from numpy.random import choice
import random as rd
import pandas as pd
import matplotlib.pyplot as plt
import copy 
import pickle
from pprint import pprint
import copy
%matplotlib inline

**Import genetic data** (linkage) as a dataframe.

In [182]:
MRCAgenotypes = pd.read_csv("./MRCAgenotypes.csv")
pprint(MRCAgenotypes[:10])

/Users/jeje/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


         marker chromosome position(b38) CC001/UncF3011 CC001/UncF3012  \
0          UNC6          1       3010274              T              T   
1   JAX00000010          1       3135418              A              A   
2   JAX00240603          1       3252796              C              C   
3   JAX00240610          1       3266608              C              C   
4   JAX00240613          1       3323400              T              T   
5   JAX00240636          1       3389563              C              C   
6   JAX00240649          1       3454984              C              C   
7   JAX00000040          1       3544819              G              G   
8  UNC010515443          1       3668628              G              G   
9       UNC9371          1       3892718              C              C   

  CC001/UncM3005 CC001/UncM3007 CC002/UncF3011 CC002/UncM3009 CC003/UncF3006  \
0              T              T              T              T              T   
1              A         

***The descendants***

In [9]:
print(list(MRCAgenotypes.columns[3:]))

['CC001/UncF3011', 'CC001/UncF3012', 'CC001/UncM3005', 'CC001/UncM3007', 'CC002/UncF3011', 'CC002/UncM3009', 'CC003/UncF3006', 'CC003/UncF3017', 'CC003/UncM3007', 'CC003/UncM3015', 'CC004/TauUncF3006', 'CC004/TauUncF3011', 'CC004/TauUncM3003', 'CC004/TauUncM3004', 'CC005/TauUncF3007', 'CC005/TauUncM3008', 'CC006/TauUncF3254', 'CC006/TauUncM3257', 'CC007/UncF3004', 'CC007/UncM3002', 'CC008/GeniUncF212', 'CC008/GeniUncM207', 'CC009/UncF255', 'CC009/UncM250', 'CC010/GeniUncF203', 'CC010/GeniUncM212', 'CC011/UncF494', 'CC011/UncM495', 'CC012/GeniUncF212', 'CC012/GeniUncM213', 'CC013/GeniUncF215', 'CC013/GeniUncF216', 'CC013/GeniUncF217', 'CC013/GeniUncF550', 'CC013/GeniUncM225', 'CC013/GeniUncM226', 'CC015/UncM551', 'CC016/GeniUncF207', 'CC016/GeniUncM201', 'CC017/UncF285', 'CC017/UncM286', 'CC018/UncF296', 'CC018/UncM298', 'CC019/TauUncF3732', 'CC019/TauUncM3730', 'CC020/GeniUncF201', 'CC020/GeniUncM208', 'CC021/UncF850', 'CC021/UncF872', 'CC021/UncF877', 'CC021/UncF905', 'CC021/UncM683',

***Deleting all non A/T/G/C data and select only one of the 2 genotypes for each founder*** : loss of 92% of the SNPs.

In [189]:
MRCAgenotypes_ch1 = MRCAgenotypes[MRCAgenotypes["chromosome"] == 1].T

In [190]:
MRCAgenotypes_ch1.T[:10]

,0,1,2,3,4,5,6,7,8,9,...,5524,5525,5526,5527,5528,5529,5530,5531,5532,5533
marker,UNC6,JAX00000010,JAX00240603,JAX00240610,JAX00240613,JAX00240636,JAX00240649,JAX00000040,UNC010515443,UNC9371,...,UNC2465770,UNC2465840,UNC2465893,UNC2465966,UNC2466002,UNC2466045,UNC2466105,UNC2466156,UNC2466203,UNC2466290
chromosome,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
position(b38),3010274,3135418,3252796,3266608,3323400,3389563,3454984,3544819,3668628,3892718,...,195230950,195241261,195253789,195269809,195276493,195288193,195300200,195315452,195332026,195360017
CC001/UncF3011,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,H,H,T,G,A,A
CC001/UncF3012,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,T,H,T,G,A,A
CC001/UncM3005,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,T,H,T,G,A,A
CC001/UncM3007,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,T,H,T,G,A,A
CC002/UncF3011,T,A,C,C,T,C,C,G,G,C,...,A,T,G,N,C,A,T,G,A,G
CC002/UncM3009,T,A,C,C,T,C,C,G,G,C,...,A,T,G,N,C,A,T,G,A,G
CC003/UncF3006,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,T,A,T,A,A,A


In [200]:
MRCAgenotypes_ch1.T[3:10]

,0,1,2,3,4,5,6,7,8,9,...,5524,5525,5526,5527,5528,5529,5530,5531,5532,5533
CC001/UncF3011,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,H,H,T,G,A,A
CC001/UncF3012,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,T,H,T,G,A,A
CC001/UncM3005,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,T,H,T,G,A,A
CC001/UncM3007,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,T,H,T,G,A,A
CC002/UncF3011,T,A,C,C,T,C,C,G,G,C,...,A,T,G,N,C,A,T,G,A,G
CC002/UncM3009,T,A,C,C,T,C,C,G,G,C,...,A,T,G,N,C,A,T,G,A,G
CC003/UncF3006,T,A,C,C,T,C,C,G,G,C,...,G,C,T,N,T,A,T,A,A,A


In [207]:
MRCAgenotypes_ch1.T[3:10].T

,CC001/UncF3011,CC001/UncF3012,CC001/UncM3005,CC001/UncM3007,CC002/UncF3011,CC002/UncM3009,CC003/UncF3006
0,T,T,T,T,T,T,T
1,A,A,A,A,A,A,A
2,C,C,C,C,C,C,C
3,C,C,C,C,C,C,C
4,T,T,T,T,T,T,T
5,C,C,C,C,C,C,C
6,C,C,C,C,C,C,C
7,G,G,G,G,G,G,G
8,G,G,G,G,G,G,G
9,C,C,C,C,C,C,C


In [208]:
np.logical_or(np.logical_or(np.logical_or(MRCAgenotypes_ch1.T[3:10].T=="A", MRCAgenotypes_ch1.T[3:10].T=="T"), MRCAgenotypes_ch1.T[3:10].T=="G"), MRCAgenotypes_ch1.T[3:10].T=="C")

,CC001/UncF3011,CC001/UncF3012,CC001/UncM3005,CC001/UncM3007,CC002/UncF3011,CC002/UncM3009,CC003/UncF3006
0,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True


In [209]:
MRCAgenotypes_ch1.T[3:10].T[np.logical_or(np.logical_or(np.logical_or(MRCAgenotypes_ch1.T[3:10].T=="A", MRCAgenotypes_ch1.T[3:10].T=="T"), MRCAgenotypes_ch1.T[3:10].T=="G"), MRCAgenotypes_ch1.T[3:10].T=="C")]

,CC001/UncF3011,CC001/UncF3012,CC001/UncM3005,CC001/UncM3007,CC002/UncF3011,CC002/UncM3009,CC003/UncF3006
0,T,T,T,T,T,T,T
1,A,A,A,A,A,A,A
2,C,C,C,C,C,C,C
3,C,C,C,C,C,C,C
4,T,T,T,T,T,T,T
5,C,C,C,C,C,C,C
6,C,C,C,C,C,C,C
7,G,G,G,G,G,G,G
8,G,G,G,G,G,G,G
9,C,C,C,C,C,C,C


In [210]:
MRCAgenotypes_ch1.T[3:10].T[np.logical_or(np.logical_or(np.logical_or(MRCAgenotypes_ch1.T[3:10].T=="A", MRCAgenotypes_ch1.T[3:10].T=="T"), MRCAgenotypes_ch1.T[3:10].T=="G"), MRCAgenotypes_ch1.T[3:10].T=="C")].dropna(how='any')

,CC001/UncF3011,CC001/UncF3012,CC001/UncM3005,CC001/UncM3007,CC002/UncF3011,CC002/UncM3009,CC003/UncF3006
0,T,T,T,T,T,T,T
1,A,A,A,A,A,A,A
2,C,C,C,C,C,C,C
3,C,C,C,C,C,C,C
4,T,T,T,T,T,T,T
5,C,C,C,C,C,C,C
6,C,C,C,C,C,C,C
7,G,G,G,G,G,G,G
8,G,G,G,G,G,G,G
9,C,C,C,C,C,C,C


In [214]:
len(MRCAgenotypes_ch1.T),len(MRCAgenotypes_ch1),len(MRCAgenotypes_ch1_reduced.T)

(199, 5534, 196)

In [224]:
MRCAgenotypes_ch1_reduced = MRCAgenotypes_ch1.T[3:].T[np.logical_or(np.logical_or(np.logical_or(MRCAgenotypes_ch1.T[3:].T=="A", MRCAgenotypes_ch1.T[3:].T=="T"), MRCAgenotypes_ch1.T[3:].T=="G"), MRCAgenotypes_ch1.T[3:].T=="C")].dropna(how='any')
len(MRCAgenotypes_ch1),len(MRCAgenotypes_ch1_reduced),(len(MRCAgenotypes_ch1)-len(MRCAgenotypes_ch1_reduced))/len(MRCAgenotypes_ch1)

(5534, 418, 0.9244669316949765)

In [212]:
MRCAgenotypes_ch1_reduced[:10]

,CC001/UncF3011,CC001/UncF3012,CC001/UncM3005,CC001/UncM3007,CC002/UncF3011,CC002/UncM3009,CC003/UncF3006,CC003/UncF3017,CC003/UncM3007,CC003/UncM3015,...,CC072/TauUncF3006,CC072/TauUncM3004,CC073/UncF472,CC073/UncM474,CC074/UncF165,CC074/UncM166,CC075/UncF415,CC075/UncM419,CC076/UncF411,CC076/UncM392
3,C,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
7,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
48,A,A,A,A,A,A,A,A,A,A,...,G,G,G,G,G,G,G,G,G,G
75,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
87,T,T,T,T,T,T,T,T,T,T,...,C,C,C,C,C,C,C,C,C,C
121,C,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
139,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
141,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
159,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
185,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G


***Transform it into SNPs with A632 as reference***

In [216]:
MRCAgenotypes_ch1_reduced.columns

Index(['CC001/UncF3011', 'CC001/UncF3012', 'CC001/UncM3005', 'CC001/UncM3007',
       'CC002/UncF3011', 'CC002/UncM3009', 'CC003/UncF3006', 'CC003/UncF3017',
       'CC003/UncM3007', 'CC003/UncM3015',
       ...
       'CC072/TauUncF3006', 'CC072/TauUncM3004', 'CC073/UncF472',
       'CC073/UncM474', 'CC074/UncF165', 'CC074/UncM166', 'CC075/UncF415',
       'CC075/UncM419', 'CC076/UncF411', 'CC076/UncM392'],
      dtype='object', length=196)

In [225]:
C = MRCAgenotypes_ch1_reduced.columns
for k in range(len(C)):
    MRCAgenotypes_ch1_reduced[k] = (MRCAgenotypes_ch1_reduced[C[k]] != MRCAgenotypes_ch1_reduced[C[0]])

MRCAgenotypes_ch1_reduced[:10]

,CC001/UncF3011,CC001/UncF3012,CC001/UncM3005,CC001/UncM3007,CC002/UncF3011,CC002/UncM3009,CC003/UncF3006,CC003/UncF3017,CC003/UncM3007,CC003/UncM3015,...,186,187,188,189,190,191,192,193,194,195
3,C,C,C,C,C,C,C,C,C,C,...,False,False,False,False,False,False,False,False,False,False
7,G,G,G,G,G,G,G,G,G,G,...,False,False,False,False,False,False,False,False,False,False
48,A,A,A,A,A,A,A,A,A,A,...,True,True,True,True,True,True,True,True,True,True
75,A,A,A,A,A,A,A,A,A,A,...,False,False,False,False,False,False,False,False,False,False
87,T,T,T,T,T,T,T,T,T,T,...,True,True,True,True,True,True,True,True,True,True
121,C,C,C,C,C,C,C,C,C,C,...,False,False,False,False,False,False,False,False,False,False
139,G,G,G,G,G,G,G,G,G,G,...,False,False,False,False,False,False,False,False,False,False
141,G,G,G,G,G,G,G,G,G,G,...,False,False,False,False,False,False,False,False,False,False
159,G,G,G,G,G,G,G,G,G,G,...,False,False,False,False,False,False,False,False,False,False
185,G,G,G,G,G,G,G,G,G,G,...,False,False,False,False,False,False,False,False,False,False


In [227]:
founders = MRCAgenotypes_ch1_reduced[range(len(C))].astype(int)
founders[:10]

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,195
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
75,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
121,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
159,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [229]:
founders.T.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 1, ..., 0, 1, 1]])

In [237]:
F_geno = founders.T.values
F_geno.shape

(196, 418)

## Calculate similarity

In [238]:
def normalized_cross_similarity (array,n,p) :
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return max(0,np.round(np.mean(B),5)*2-1)    # Normalize between 0 and 1

In [239]:
def similarity_standard_deviation(array,n,p):
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return np.round(np.sqrt(np.var(B)),3)

In [240]:
n, p = F_geno.shape

In [241]:
normalized_cross_similarity (F_geno,n,p)

0.62602

In [242]:
similarity_standard_deviation(F_geno,n,p)

0.067

# MAGIC Maize

https://figshare.com/articles/MAGIC_Maize_founders_genotyping/1437453

https://genomebiology.biomedcentral.com/articles/10.1186/s13059-015-0716-z#Sec12

## Prepare data

In [2]:
import numpy as np
from numpy.random import choice
import random as rd
import pandas as pd
import matplotlib.pyplot as plt
import copy 
import pickle
from pprint import pprint
import copy
%matplotlib inline

**Import genetic data** (linkage) as a dataframe.

In [39]:
MMfounders = pd.read_csv("./MMfounders.geno.csv", delimiter=" ")
pprint(MMfounders.T[:10])

                 0       1    2        3    4      5      6        7   8   \
LINE           A632  A632.1  B73  B73.ref  B96  B96.1  CML91  CML91.1  F7   
SYN83             C       C    T        T    T      T      H        H   C   
SYN79             C       C    C        C    C      C      C        C   C   
PZE.101000060     C       C    C        C    C      C      C        C   C   
PZE.101000088     G       G    G        G    G      G      G        G   G   
PZE.101000083     C       C    G        G    G      G      G        G   C   
PZE.101000108     G       G    G        G    G      G      G        G   G   
PZE.101000111     C       C    C        C    C      C      C        C   C   
PZE.101000121     N       N    N        N    N      N      N        N   N   
PZE.101000122     N       N    N        N    N      N      N        N   N   

                 9    10     11     12       13    14      15     16       17  
LINE           F7.1  H99  H99.1  HP301  HP301.1  Mo17  Mo17.1  W153R  W1

In [45]:
MMfounders.T.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,54235,54235,54235,54235,54235,54235,54235,54235,54235,54235,54235,54235,54235,54235,54235,54235,54235,54235
unique,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
top,C,C,C,C,C,C,H,H,C,C,C,C,C,C,G,G,C,C
freq,12678,12702,12695,12666,12757,12756,12960,12947,12884,12884,12816,12828,12780,12780,12542,12541,11217,11216


***The 9 founders*** (some sequenced twice)

In [40]:
print(list(MMfounders["LINE"]))

['A632', 'A632.1', 'B73', 'B73.ref', 'B96', 'B96.1', 'CML91', 'CML91.1', 'F7', 'F7.1', 'H99', 'H99.1', 'HP301', 'HP301.1', 'Mo17', 'Mo17.1', 'W153R', 'W153R.1']


***Deleting all non A/T/G/C data and select only one of the 2 genotypes for each founder*** : loss of 55% of the SNPs.

In [72]:
MMfounders.T[1:10]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
SYN83,C,C,T,T,T,T,H,H,C,C,T,T,C,C,C,C,T,T
SYN79,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C
PZE.101000060,C,C,C,C,C,C,C,C,C,C,C,C,A,A,C,C,C,C
PZE.101000088,G,G,G,G,G,G,G,G,G,G,N,N,G,G,C,C,G,G
PZE.101000083,C,C,G,G,G,G,G,G,C,C,G,G,C,C,N,N,G,G
PZE.101000108,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G
PZE.101000111,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,T,T
PZE.101000121,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N
PZE.101000122,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N


In [73]:
np.logical_or(np.logical_or(np.logical_or(MMfounders.T[1:10]=="A", MMfounders.T[1:10]=="T"), MMfounders.T[1:10]=="G"), MMfounders.T[1:10]=="C")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
SYN83,True,True,True,True,True,True,False,False,True,True,True,True,True,True,True,True,True,True
SYN79,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
PZE.101000060,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
PZE.101000088,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,True,True,True
PZE.101000083,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True
PZE.101000108,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
PZE.101000111,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
PZE.101000121,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
PZE.101000122,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [85]:
MMfounders.T[1:10][np.logical_or(np.logical_or(np.logical_or(MMfounders.T[1:10]=="A", MMfounders.T[1:10]=="T"), MMfounders.T[1:10]=="G"), MMfounders.T[1:10]=="C")]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
SYN83,C,C,T,T,T,T,NaN,NaN,C,C,T,T,C,C,C,C,T,T
SYN79,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C
PZE.101000060,C,C,C,C,C,C,C,C,C,C,C,C,A,A,C,C,C,C
PZE.101000088,G,G,G,G,G,G,G,G,G,G,NaN,NaN,G,G,C,C,G,G
PZE.101000083,C,C,G,G,G,G,G,G,C,C,G,G,C,C,NaN,NaN,G,G
PZE.101000108,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G
PZE.101000111,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,T,T
PZE.101000121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PZE.101000122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
MMfounders.T[1:10][np.logical_or(np.logical_or(np.logical_or(MMfounders.T[1:10]=="A", MMfounders.T[1:10]=="T"), MMfounders.T[1:10]=="G"), MMfounders.T[1:10]=="C")].dropna(how='any')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
SYN79,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C
PZE.101000060,C,C,C,C,C,C,C,C,C,C,C,C,A,A,C,C,C,C
PZE.101000108,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G
PZE.101000111,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,T,T


In [102]:
MMfounders.T[:1]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
LINE,A632,A632.1,B73,B73.ref,B96,B96.1,CML91,CML91.1,F7,F7.1,H99,H99.1,HP301,HP301.1,Mo17,Mo17.1,W153R,W153R.1


In [124]:
MMfounders_reduced = MMfounders.T[1:][np.logical_or(np.logical_or(np.logical_or(MMfounders.T[1:]=="A", MMfounders.T[1:]=="T"), MMfounders.T[1:]=="G"), MMfounders.T[1:]=="C")].dropna(how='any')[range(0,18,2)]
len(MMfounders.T),len(MMfounders_reduced),(len(MMfounders.T)-len(MMfounders_reduced))/len(MMfounders.T)

(54235, 24284, 0.5522448603300452)

In [125]:
MMfounders_reduced[:10]

,0,2,4,6,8,10,12,14,16
SYN79,C,C,C,C,C,C,C,C,C
PZE.101000060,C,C,C,C,C,C,A,C,C
PZE.101000108,G,G,G,G,G,G,G,G,G
PZE.101000111,C,C,C,C,C,C,C,C,T
PZE.101000301,A,G,G,G,G,A,A,A,G
PZE.101000344,T,C,C,C,C,T,T,T,C
PZE.101000359,C,C,C,C,C,C,C,C,C
PZE.101000424,C,A,A,A,C,A,A,A,A
PZE.101000431,G,C,C,C,G,G,G,G,C
PZE.101000442,G,T,G,G,G,G,G,G,T


***Transform it into SNPs with A632 as reference***

In [139]:
MMfounders_reduced[2] == MMfounders_reduced[0]

SYN79                      True
PZE.101000060              True
PZE.101000108              True
PZE.101000111              True
PZE.101000301             False
PZE.101000344             False
PZE.101000359              True
PZE.101000424             False
PZE.101000431             False
PZE.101000442             False
PZE.101000449              True
PZE.101000450              True
PZE.101000451              True
PZE.101000530             False
PZE.101000557              True
PZE.101000659              True
PZE.101000673             False
PZE.101000740             False
PZE.101000754             False
SYN9660                    True
SYN10558                   True
PZE.101001103              True
PZE.101001107              True
PZE.101001108              True
PZE.101001110              True
SYN10560                   True
PZE.101001114              True
PZE.101001115              True
PZE.101001116              True
SYN13279                   True
                          ...  
PZE.1101

In [145]:
for k in range(1,18,2):
    MMfounders_reduced[k] = (MMfounders_reduced[k-1] != MMfounders_reduced[0])

MMfounders_reduced[:10]

,0,2,4,6,8,10,12,14,16,1,3,5,7,9,11,13,15,17
SYN79,C,C,C,C,C,C,C,C,C,False,False,False,False,False,False,False,False,False
PZE.101000060,C,C,C,C,C,C,A,C,C,False,False,False,False,False,False,True,False,False
PZE.101000108,G,G,G,G,G,G,G,G,G,False,False,False,False,False,False,False,False,False
PZE.101000111,C,C,C,C,C,C,C,C,T,False,False,False,False,False,False,False,False,True
PZE.101000301,A,G,G,G,G,A,A,A,G,False,True,True,True,True,False,False,False,True
PZE.101000344,T,C,C,C,C,T,T,T,C,False,True,True,True,True,False,False,False,True
PZE.101000359,C,C,C,C,C,C,C,C,C,False,False,False,False,False,False,False,False,False
PZE.101000424,C,A,A,A,C,A,A,A,A,False,True,True,True,False,True,True,True,True
PZE.101000431,G,C,C,C,G,G,G,G,C,False,True,True,True,False,False,False,False,True
PZE.101000442,G,T,G,G,G,G,G,G,T,False,True,False,False,False,False,False,False,True


In [156]:
founders = MMfounders_reduced[range(1,18,2)].astype(int)*1
founders[:10].T

,SYN79,PZE.101000060,PZE.101000108,PZE.101000111,PZE.101000301,PZE.101000344,PZE.101000359,PZE.101000424,PZE.101000431,PZE.101000442
1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,1,0,1,1,1
5,0,0,0,0,1,1,0,1,1,0
7,0,0,0,0,1,1,0,1,1,0
9,0,0,0,0,1,1,0,0,0,0
11,0,0,0,0,0,0,0,1,0,0
13,0,1,0,0,0,0,0,1,0,0
15,0,0,0,0,0,0,0,1,0,0
17,0,0,0,1,1,1,0,1,1,1


In [163]:
founders.T.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [165]:
F_geno = founders.T.values
F_geno.shape

(9, 24284)

## Calculate similarity

In [166]:
def normalized_cross_similarity (array,n,p) :
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return max(0,np.round(np.mean(B),5)*2-1)    # Normalize between 0 and 1

In [167]:
def similarity_standard_deviation(array,n,p):
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return np.round(np.sqrt(np.var(B)),3)

In [168]:
n, p = F_geno.shape

In [169]:
normalized_cross_similarity (F_geno,n,p)

0.38764

In [170]:
similarity_standard_deviation(F_geno,n,p)

0.102

# MAGIC Arabidopsis thaliana

https://www.genetics.org/content/198/4/1751#sec-1

http://mtweb.cs.ucl.ac.uk/mus/www/19genomes/magic.html

http://mtweb.cs.ucl.ac.uk/mus/www/19genomes/variants.tables/

## Prepare data

In [2]:
import numpy as np
from numpy.random import choice
import random as rd
import pandas as pd
import matplotlib.pyplot as plt
import copy 
import pickle
from pprint import pprint
import copy
%matplotlib inline

**Import genetic data** (linkage) as a dataframe.

In [9]:
chr1_alleles = pd.read_csv("./chr1.alleles.txt", delimiter="\t").T
pprint(chr1_alleles.T[:10])

  chr pse.bp    bp nalleles maf  \
0   1     11    11        2  18   
1   1     18    18        2  18   
2   1     22  21.5        3  16   
3   1     38  29.5        9  10   
4   1    136  57.5        2  18   
5   1    151  58.5        2  18   
6   1    170  61.5        5  15   
7   1    240    80        2  18   
8   1    243    83        2  18   
9   1    251    91        4  13   

                                               bur-0                    can-0  \
0                                                  T                        T   
1                                                  T                        T   
2                                                  0                        0   
3  CTCTGAATCCTTAATCCCTAATCCCAAAATCCCTAAATCACTTAAT...  CTCTGAATCCTTAATCCCTAAAT   
4                                                  0                        0   
5                                                  0                        0   
6         CCTATACCCTAAACCCTAAACCCAAAAATCTTTAAAT

In [10]:
chr1_alleles.T.describe()

,chr,pse.bp,bp,nalleles,maf,bur-0,can-0,col-0,ct-1,edi-0,...,no-0,oy-0,po-0,rsch-4,sf-2,tsu-0,wil-2,ws-0,wu-0,zu-0
count,811030,811030,811030.0,811030,811030,811030,811030,811030,811030,811030,...,811030,811030,811030,811030,811030,811030,811030,811030,811030,811030
unique,1,811030,811024.0,18,18,27249,27847,25522,25851,27534,...,27304,25684,26408,26849,27631,27469,27496,27380,26917,26951
top,1,19994764,27097059.5,2,18,T,A,A,A,A,...,A,A,A,A,A,A,A,T,A,A
freq,811030,1,2.0,661208,335421,176448,176455,179262,177090,177015,...,176846,176793,159648,176968,176410,177017,177033,176545,176745,177216


***The 19 founders***

In [14]:
print(list(chr1_alleles.T.columns[5:]))

['bur-0', 'can-0', 'col-0', 'ct-1', 'edi-0', 'hi-0', 'kn-0', 'ler-0', 'mt-0', 'no-0', 'oy-0', 'po-0', 'rsch-4', 'sf-2', 'tsu-0', 'wil-2', 'ws-0', 'wu-0', 'zu-0']


***Deleting all non A/T/G/C data and select only one of the 2 genotypes for each founder*** : loss of 39% of the SNPs.

In [19]:
chr1_alleles[5:].T[:10]

,bur-0,can-0,col-0,ct-1,edi-0,hi-0,kn-0,ler-0,mt-0,no-0,oy-0,po-0,rsch-4,sf-2,tsu-0,wil-2,ws-0,wu-0,zu-0
0,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T,T,T,T
1,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T,T,T,T
2,0,0,0,T,0,0,0,0,0,0,T,TCCTAAAT,0,0,0,0,0,0,0
3,CTCTGAATCCTTAATCCCTAATCCCAAAATCCCTAAATCACTTAAT...,CTCTGAATCCTTAATCCCTAAAT,CTCTGAATCCTTAATCCCTAAAT,CTCTGGATCCTTAATCCTTAATCCCTAAAT,CTCTGAATCCTTAATCCCTAAATCTATAAAT,CTCTGAATCCTTAAT,CTATAAATCTCTGAATCCTTAATCCCTAATCCCAAAATCCCTAAAT...,CTCTGAATCCTTAATCCCTAAAT,CTCTGAATCCTTAATCCCTAAAT,CTCTGAATCCTTAATCCCTAAAT,CTCTGGATCCTTAATCCTTAATCCCTAAAT,CTCTGGATCCTTAATCCCTAATCCCAAAATCCCTAAATCACTTAAT...,CTCTGAATCCTTAATCCCTAAAT,CTCTGAATCCTTAATCCTAAAT,CTCTGAATCCTTAATCCCTAAAT,CTCTGAATCCTTAATCCCTAAAT,CTCTGAATCCTTAATCCCTAAAT,0,CTCTGAATCCTTAATCCCTAAAT
4,0,0,0,0,0,0,0,0,0,0,0,AATACTTAAATCCC,0,0,0,0,0,0,0
5,0,0,0,0,0,0,ATACTTAAATCCCTAG,0,0,0,0,0,0,0,0,0,0,0,0
6,CCTATACCCTAAACCCTAAACCCAAAAATCTTTAAATCCTAA,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,TTTA,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,CCTTAATCCTTAATCCTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAC,TTTAAATCCTAA
7,G,G,G,G,0,G,G,G,G,G,G,G,G,G,G,G,G,G,G
8,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T
9,TACCTAAT,0,TACCTAAT,TACCTAAT,TACCTAAA,TACCTAAT,TACCTAAT,TACCTAAT,TACCTAAT,TACCTAAT,TACCTAAT,TCCCTAAT,TACCTAAT,TACCTAAT,TACCTAAA,TACCTAAT,TACCTAAT,TCCCTAAT,TCCCTAAT


In [20]:
np.logical_or(np.logical_or(np.logical_or(chr1_alleles[5:].T[:10]=="A", chr1_alleles[5:].T[:10]=="T"), chr1_alleles[5:].T[:10]=="G"), chr1_alleles[5:].T[:10]=="C")

,bur-0,can-0,col-0,ct-1,edi-0,hi-0,kn-0,ler-0,mt-0,no-0,oy-0,po-0,rsch-4,sf-2,tsu-0,wil-2,ws-0,wu-0,zu-0
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [21]:
chr1_alleles[5:].T[:10][np.logical_or(np.logical_or(np.logical_or(chr1_alleles[5:].T[:10]=="A", chr1_alleles[5:].T[:10]=="T"), chr1_alleles[5:].T[:10]=="G"), chr1_alleles[5:].T[:10]=="C")]

,bur-0,can-0,col-0,ct-1,edi-0,hi-0,kn-0,ler-0,mt-0,no-0,oy-0,po-0,rsch-4,sf-2,tsu-0,wil-2,ws-0,wu-0,zu-0
0,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T,T,T,T
1,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T,T,T,T
2,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,G,G,G,G,NaN,G,G,G,G,G,G,G,G,G,G,G,G,G,G
8,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
chr1_alleles[5:].T[:10][np.logical_or(np.logical_or(np.logical_or(chr1_alleles[5:].T[:10]=="A", chr1_alleles[5:].T[:10]=="T"), chr1_alleles[5:].T[:10]=="G"), chr1_alleles[5:].T[:10]=="C")].dropna(how='any')

,bur-0,can-0,col-0,ct-1,edi-0,hi-0,kn-0,ler-0,mt-0,no-0,oy-0,po-0,rsch-4,sf-2,tsu-0,wil-2,ws-0,wu-0,zu-0
0,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T,T,T,T
1,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T,T,T,T
8,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T


In [25]:
chr1_alleles_reduced = chr1_alleles[5:].T[np.logical_or(np.logical_or(np.logical_or(chr1_alleles[5:].T=="A", chr1_alleles[5:].T=="T"), chr1_alleles[5:].T=="G"), chr1_alleles[5:].T=="C")].dropna(how='any')
len(chr1_alleles.T),len(chr1_alleles_reduced),(len(chr1_alleles.T)-len(chr1_alleles_reduced))/len(chr1_alleles.T)

(811030, 497496, 0.38658742586587425)

In [26]:
chr1_alleles_reduced[:10]

,bur-0,can-0,col-0,ct-1,edi-0,hi-0,kn-0,ler-0,mt-0,no-0,oy-0,po-0,rsch-4,sf-2,tsu-0,wil-2,ws-0,wu-0,zu-0
0,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T,T,T,T
1,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T,T,T,T
8,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,C,T
12,T,G,G,T,T,G,T,G,G,G,T,T,G,T,G,G,G,G,G
13,T,C,C,T,C,C,T,C,C,C,T,T,C,T,C,C,C,C,C
14,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,G,T
15,C,T,T,C,T,T,C,T,T,T,C,C,T,C,T,T,T,T,T
22,T,T,T,T,A,T,T,T,T,T,T,T,T,T,T,T,T,T,T
27,G,G,G,G,A,G,G,G,G,G,G,G,G,G,G,G,G,G,G
29,A,G,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A


***Transform it into SNPs with A632 as reference***

In [27]:
C = chr1_alleles_reduced.columns
for k in range(len(C)):
    chr1_alleles_reduced[k] = (chr1_alleles_reduced[C[k]] != chr1_alleles_reduced[C[0]])

chr1_alleles_reduced[:10]

,bur-0,can-0,col-0,ct-1,edi-0,hi-0,kn-0,ler-0,mt-0,no-0,...,9,10,11,12,13,14,15,16,17,18
0,T,T,T,T,T,T,T,T,T,T,...,False,False,False,False,False,True,False,False,False,False
1,T,T,T,T,T,T,T,T,T,T,...,False,False,False,False,False,True,False,False,False,False
8,T,T,T,T,T,T,T,T,T,T,...,False,False,False,False,False,False,False,False,True,False
12,T,G,G,T,T,G,T,G,G,G,...,True,False,False,True,False,True,True,True,True,True
13,T,C,C,T,C,C,T,C,C,C,...,True,False,False,True,False,True,True,True,True,True
14,G,G,G,G,G,G,G,G,G,G,...,False,False,False,False,False,False,False,False,False,True
15,C,T,T,C,T,T,C,T,T,T,...,True,False,False,True,False,True,True,True,True,True
22,T,T,T,T,A,T,T,T,T,T,...,False,False,False,False,False,False,False,False,False,False
27,G,G,G,G,A,G,G,G,G,G,...,False,False,False,False,False,False,False,False,False,False
29,A,G,A,A,A,A,A,A,A,A,...,False,False,False,False,False,False,False,False,False,False


In [28]:
founders = chr1_alleles_reduced[range(len(C))].astype(int)
founders[:10]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
12,0,1,1,0,0,1,0,1,1,1,0,0,1,0,1,1,1,1,1
13,0,1,1,0,1,1,0,1,1,1,0,0,1,0,1,1,1,1,1
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15,0,1,1,0,1,1,0,1,1,1,0,0,1,0,1,1,1,1,1
22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
founders.T.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [30]:
F_geno = founders.T.values
F_geno.shape

(19, 497496)

## Calculate similarity

In [33]:
def normalized_cross_similarity (array,n,p) :
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return max(0,np.round(np.mean(B),5)*2-1)    # Normalize between 0 and 1

In [34]:
def similarity_standard_deviation(array,n,p):
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return np.round(np.sqrt(np.var(B)),3)

In [35]:
n, p = F_geno.shape

In [36]:
normalized_cross_similarity (F_geno,n,p)

0.5548

In [37]:
similarity_standard_deviation(F_geno,n,p)

0.031

# DGRP2 Drosophila

http://dgrp2.gnets.ncsu.edu/data.html

## Prepare data

In [1]:
import numpy as np
from numpy.random import choice
import random as rd
import pandas as pd
import matplotlib.pyplot as plt
import copy 
import pickle
from pprint import pprint
import copy
%matplotlib inline

**Import genetic data** (linkage) as a dataframe.

In [5]:
DGRP2 = pd.read_csv("./dgrp2.tgeno.txt", delimiter=" ", nrows=10000).T
pprint(DGRP2.T[:10])

  chr   pos           id ref alt refc altc qual cov line_21  ... line_887  \
0  2L  4998  2L_4998_SNP   G   A  117    5  999  12       0  ...        0   
1  2L  5002  2L_5002_SNP   G   T  127    1  999  13       0  ...        0   
2  2L  5039  2L_5039_SNP   C   T    1  118  999  21       2  ...        2   
3  2L  5040  2L_5040_SNP   G   A    1  118  999  21       2  ...        2   
4  2L  5092  2L_5092_SNP   C   T    6  119  999  22       2  ...        2   
5  2L  5095  2L_5095_SNP   T   A    4  115  999  22       2  ...        2   
6  2L  5153  2L_5153_SNP   A   C  155    2  999  14       0  ...        0   
7  2L  5232  2L_5232_SNP   C   T  191    1  999  19       0  ...        0   
8  2L  5233  2L_5233_SNP   G   C  189    2  999  19       0  ...        0   
9  2L  5317  2L_5317_SNP   G   A  177   11  999  17       0  ...        0   

  line_890 line_892 line_894 line_897 line_900 line_907 line_908 line_911  \
0        -        2        -        0        0        0        -        0  

In [7]:
DGRP2.T[:10]

,chr,pos,id,ref,alt,refc,altc,qual,cov,line_21,...,line_887,line_890,line_892,line_894,line_897,line_900,line_907,line_908,line_911,line_913
0,2L,4998,2L_4998_SNP,G,A,117,5,999,12,0,...,0,-,2,-,0,0,0,-,0,0
1,2L,5002,2L_5002_SNP,G,T,127,1,999,13,0,...,0,-,0,-,0,0,0,-,0,0
2,2L,5039,2L_5039_SNP,C,T,1,118,999,21,2,...,2,-,2,-,2,2,-,2,2,2
3,2L,5040,2L_5040_SNP,G,A,1,118,999,21,2,...,2,-,2,-,2,2,-,2,2,2
4,2L,5092,2L_5092_SNP,C,T,6,119,999,22,2,...,2,-,2,-,2,2,-,2,2,2
5,2L,5095,2L_5095_SNP,T,A,4,115,999,22,2,...,2,-,2,-,2,2,-,2,2,2
6,2L,5153,2L_5153_SNP,A,C,155,2,999,14,0,...,0,0,0,0,0,0,-,0,0,0
7,2L,5232,2L_5232_SNP,C,T,191,1,999,19,0,...,0,0,0,0,0,0,0,0,0,0
8,2L,5233,2L_5233_SNP,G,C,189,2,999,19,0,...,0,0,0,0,0,0,0,0,0,0
9,2L,5317,2L_5317_SNP,G,A,177,11,999,17,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
DGRP2.T.describe()

,chr,pos,id,ref,alt,refc,altc,qual,cov,line_21,...,line_887,line_890,line_892,line_894,line_897,line_900,line_907,line_908,line_911,line_913
count,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
unique,1,9988,10000,506,213,205,203,1,64,3,...,3,3,3,3,3,3,3,3,3,3
top,2L,477871,2L_258080_SNP,C,T,204,1,999,34,0,...,0,0,0,0,0,0,0,0,0,0
freq,10000,2,1,2640,3057,1629,3166,10000,834,8862,...,8318,8797,8884,8246,8733,8952,8247,8921,8766,8483


***The descendants***

In [9]:
print(list(DGRP2.T.columns[9:]))

['line_21', 'line_26', 'line_28', 'line_31', 'line_32', 'line_38', 'line_40', 'line_41', 'line_42', 'line_45', 'line_48', 'line_49', 'line_57', 'line_59', 'line_69', 'line_73', 'line_75', 'line_83', 'line_85', 'line_88', 'line_91', 'line_93', 'line_100', 'line_101', 'line_105', 'line_109', 'line_129', 'line_136', 'line_138', 'line_142', 'line_149', 'line_153', 'line_158', 'line_161', 'line_176', 'line_177', 'line_181', 'line_189', 'line_195', 'line_208', 'line_217', 'line_223', 'line_227', 'line_228', 'line_229', 'line_233', 'line_235', 'line_237', 'line_239', 'line_256', 'line_280', 'line_287', 'line_301', 'line_303', 'line_304', 'line_306', 'line_307', 'line_309', 'line_310', 'line_313', 'line_315', 'line_317', 'line_318', 'line_319', 'line_320', 'line_321', 'line_324', 'line_325', 'line_332', 'line_335', 'line_336', 'line_338', 'line_340', 'line_348', 'line_350', 'line_352', 'line_354', 'line_355', 'line_356', 'line_357', 'line_358', 'line_359', 'line_360', 'line_361', 'line_362', '

In [19]:
DGRP2.T.columns[9:]

Index(['line_21', 'line_26', 'line_28', 'line_31', 'line_32', 'line_38',
       'line_40', 'line_41', 'line_42', 'line_45',
       ...
       'line_887', 'line_890', 'line_892', 'line_894', 'line_897', 'line_900',
       'line_907', 'line_908', 'line_911', 'line_913'],
      dtype='object', length=205)

In [34]:
print(list(DGRP2.T.columns[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]]))

['line_73', 'line_83', 'line_93', 'line_153', 'line_223', 'line_233', 'line_303', 'line_313', 'line_373', 'line_383', 'line_443', 'line_513', 'line_563', 'line_703', 'line_783', 'line_843', 'line_853', 'line_913']


***Deleting all non A/T/G/C data and select only one of the 2 genotypes for each founder*** : loss of 45% of the SNPs.

In [35]:
DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][:10]

,line_73,line_83,line_93,line_153,line_223,line_233,line_303,line_313,line_373,line_383,line_443,line_513,line_563,line_703,line_783,line_843,line_853,line_913
0,0,0,0,0,0,-,0,-,0,-,0,-,0,-,2,-,0,0
1,0,0,0,0,0,-,0,-,0,-,0,-,0,-,0,-,0,0
2,2,2,2,2,2,2,2,-,2,-,2,-,-,-,2,-,2,2
3,2,2,2,2,2,2,2,-,2,-,2,-,-,-,2,-,2,2
4,2,2,2,2,2,2,2,-,2,-,2,-,-,-,2,-,2,2
5,2,2,2,-,2,2,2,-,2,-,2,-,-,-,2,-,2,2
6,0,0,-,0,0,0,0,0,0,-,0,0,0,0,-,0,0,0
7,0,0,0,0,0,0,0,0,0,-,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,-,0,0,0,0,0,0,0,0
9,0,0,0,0,0,-,0,0,0,-,0,0,0,0,0,0,0,0


In [36]:
np.logical_or(DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][:10]=="0", DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][:10]=="2")

,line_73,line_83,line_93,line_153,line_223,line_233,line_303,line_313,line_373,line_383,line_443,line_513,line_563,line_703,line_783,line_843,line_853,line_913
0,True,True,True,True,True,False,True,False,True,False,True,False,True,False,True,False,True,True
1,True,True,True,True,True,False,True,False,True,False,True,False,True,False,True,False,True,True
2,True,True,True,True,True,True,True,False,True,False,True,False,False,False,True,False,True,True
3,True,True,True,True,True,True,True,False,True,False,True,False,False,False,True,False,True,True
4,True,True,True,True,True,True,True,False,True,False,True,False,False,False,True,False,True,True
5,True,True,True,False,True,True,True,False,True,False,True,False,False,False,True,False,True,True
6,True,True,False,True,True,True,True,True,True,False,True,True,True,True,False,True,True,True
7,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True
9,True,True,True,True,True,False,True,True,True,False,True,True,True,True,True,True,True,True


In [37]:
DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][:10][np.logical_or(DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][:10]=="0", DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][:10]=="2")]

,line_73,line_83,line_93,line_153,line_223,line_233,line_303,line_313,line_373,line_383,line_443,line_513,line_563,line_703,line_783,line_843,line_853,line_913
0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,2,NaN,0,0
1,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0
2,2,2,2,2,2,2,2,NaN,2,NaN,2,NaN,NaN,NaN,2,NaN,2,2
3,2,2,2,2,2,2,2,NaN,2,NaN,2,NaN,NaN,NaN,2,NaN,2,2
4,2,2,2,2,2,2,2,NaN,2,NaN,2,NaN,NaN,NaN,2,NaN,2,2
5,2,2,2,NaN,2,2,2,NaN,2,NaN,2,NaN,NaN,NaN,2,NaN,2,2
6,0,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,NaN,0,0,0
7,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0
9,0,0,0,0,0,NaN,0,0,0,NaN,0,0,0,0,0,0,0,0


In [44]:
DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][:50][np.logical_or(DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][:50]=="0", DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][:50]=="2")].dropna(how='any')

,line_73,line_83,line_93,line_153,line_223,line_233,line_303,line_313,line_373,line_383,line_443,line_513,line_563,line_703,line_783,line_843,line_853,line_913
34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
DGRP2_reduced = DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns][np.logical_or(DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns]=="0", DGRP2[9:].T[DGRP2[9:][[name[-1]=="3" for name in DGRP2.T.columns[9:]]].T.columns]=="2")].dropna(how='any')
len(DGRP2.T),len(DGRP2_reduced),(len(DGRP2.T)-len(DGRP2_reduced))/len(DGRP2.T)

(10000, 5514, 0.4486)

In [46]:
DGRP2_reduced[:10]

,line_73,line_83,line_93,line_153,line_223,line_233,line_303,line_313,line_373,line_383,line_443,line_513,line_563,line_703,line_783,line_843,line_853,line_913
34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
founders = DGRP2_reduced.astype(int)//2
founders[:10]

,line_73,line_83,line_93,line_153,line_223,line_233,line_303,line_313,line_373,line_383,line_443,line_513,line_563,line_703,line_783,line_843,line_853,line_913
34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
founders.T.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [50]:
F_geno = founders.T.values
F_geno.shape

(18, 5514)

## Calculate similarity

In [51]:
def normalized_cross_similarity (array,n,p) :
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return max(0,np.round(np.mean(B),5)*2-1)    # Normalize between 0 and 1

In [52]:
def similarity_standard_deviation(array,n,p):
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return np.round(np.sqrt(np.var(B)),3)

In [53]:
n, p = F_geno.shape

In [54]:
normalized_cross_similarity (F_geno,n,p)

0.91134

In [55]:
similarity_standard_deviation(F_geno,n,p)

0.024

# DSPR Drosophila

http://wfitch.bio.uci.edu/~dspr/Data/index.html

## Prepare data

In [1]:
import numpy as np
from numpy.random import choice
import random as rd
import pandas as pd
import matplotlib.pyplot as plt
import copy 
import pickle
from pprint import pprint
import copy
%matplotlib inline

**Import genetic data** (linkage) as a dataframe.

In [57]:
snptable = pd.read_csv("./final_snptable_foundersonly.txt", delimiter="\t").T
pprint(snptable.T[:10])

  CHROM    POS REF ALT freq_A1 N_A1 freq_A2 N_A2 freq_A3 N_A3  ... freq_B3  \
0     X  11630   G   T       0   33       0   23       0   30  ...    0.04   
1     X  11637   T   A       0   35       0   32       0   40  ...    0.02   
2     X  11656   T   A       1   49       1   55       1   59  ...       1   
3     X  11698   C   T    0.06   34    0.04   25       0   39  ...       0   
4     X  11792   A   G       0   16    0.04   26       0   14  ...       0   
5     X  11807   A   G       0   10    0.12   26       0   17  ...    0.02   
6     X  11818   A   T       0   16       0   23       0   19  ...       0   
7     X  11822   C   T       0   16       0   25       0   20  ...       0   
8     X  12140   A   T    0.51   59    0.76  196       0   36  ...    0.53   
9     X  12185   C   T    0.04   28       0   78       0   43  ...    0.01   

  N_B3 freq_B4 N_B4 freq_B5 N_B5 freq_B6 N_B6 freq_B7 N_B7  
0   49       0   21       0   26    0.12   34    0.93  276  
1   61       0   29

In [59]:
snptable.T[:10]

,CHROM,POS,REF,ALT,freq_A1,N_A1,freq_A2,N_A2,freq_A3,N_A3,...,freq_B3,N_B3,freq_B4,N_B4,freq_B5,N_B5,freq_B6,N_B6,freq_B7,N_B7
0,X,11630,G,T,0,33,0,23,0,30,...,0.04,49,0,21,0,26,0.12,34,0.93,276
1,X,11637,T,A,0,35,0,32,0,40,...,0.02,61,0,29,0,32,0.11,37,0.89,316
2,X,11656,T,A,1,49,1,55,1,59,...,1,94,0.77,44,1,59,1,46,1,363
3,X,11698,C,T,0.06,34,0.04,25,0,39,...,0,51,0.1,29,0,30,0.08,12,0.8,220
4,X,11792,A,G,0,16,0.04,26,0,14,...,0,35,0.06,17,0,30,0,21,0.84,209
5,X,11807,A,G,0,10,0.12,26,0,17,...,0.02,44,0.04,26,0,36,0,19,0.91,220
6,X,11818,A,T,0,16,0,23,0,19,...,0,38,0,24,0,24,0,16,0.97,217
7,X,11822,C,T,0,16,0,25,0,20,...,0,39,0,23,0,27,0,22,0.97,212
8,X,12140,A,T,0.51,59,0.76,196,0,36,...,0.53,132,0.64,85,0.02,50,0.49,79,0.83,248
9,X,12185,C,T,0.04,28,0,78,0,43,...,0.01,97,0,47,0,62,0.04,49,0.61,243


In [60]:
snptable.T.describe()

,CHROM,POS,REF,ALT,freq_A1,N_A1,freq_A2,N_A2,freq_A3,N_A3,...,freq_B3,N_B3,freq_B4,N_B4,freq_B5,N_B5,freq_B6,N_B6,freq_B7,N_B7
count,1541300,1541300,1541300,1541300,1541300.0,1541300,1541300.0,1541300,1541300.0,1541300,...,1541300.0,1541300,1541300.0,1541300,1540139.0,1541300,1541300.0,1541300,1541300.0,1541300
unique,5,1504145,4,4,101.0,235,101.0,312,101.0,276,...,101.0,321,101.0,290,101.0,332,101.0,415,101.0,367
top,3R,2592569,G,A,1.0,51,1.0,62,1.0,63,...,1.0,67,1.0,59,1.0,65,1.0,70,1.0,67
freq,358871,4,404083,453291,1123858.0,37411,1045915.0,35119,1105476.0,32079,...,1068347.0,31452,1060078.0,34842,1069624.0,32989,1032760.0,29681,1079993.0,24084


In [97]:
snptable_X = snptable.T[snptable.T["CHROM"]=="X"].T
snptable_X.T[:10]

,CHROM,POS,REF,ALT,freq_A1,N_A1,freq_A2,N_A2,freq_A3,N_A3,...,freq_B3,N_B3,freq_B4,N_B4,freq_B5,N_B5,freq_B6,N_B6,freq_B7,N_B7
0,X,11630,G,T,0,33,0,23,0,30,...,0.04,49,0,21,0,26,0.12,34,0.93,276
1,X,11637,T,A,0,35,0,32,0,40,...,0.02,61,0,29,0,32,0.11,37,0.89,316
2,X,11656,T,A,1,49,1,55,1,59,...,1,94,0.77,44,1,59,1,46,1,363
3,X,11698,C,T,0.06,34,0.04,25,0,39,...,0,51,0.1,29,0,30,0.08,12,0.8,220
4,X,11792,A,G,0,16,0.04,26,0,14,...,0,35,0.06,17,0,30,0,21,0.84,209
5,X,11807,A,G,0,10,0.12,26,0,17,...,0.02,44,0.04,26,0,36,0,19,0.91,220
6,X,11818,A,T,0,16,0,23,0,19,...,0,38,0,24,0,24,0,16,0.97,217
7,X,11822,C,T,0,16,0,25,0,20,...,0,39,0,23,0,27,0,22,0.97,212
8,X,12140,A,T,0.51,59,0.76,196,0,36,...,0.53,132,0.64,85,0.02,50,0.49,79,0.83,248
9,X,12185,C,T,0.04,28,0,78,0,43,...,0.01,97,0,47,0,62,0.04,49,0.61,243


***The descendants***

In [98]:
print(list(snptable_X.T.columns[4:]))

['freq_A1', 'N_A1', 'freq_A2', 'N_A2', 'freq_A3', 'N_A3', 'freq_A4', 'N_A4', 'freq_A5', 'N_A5', 'freq_A6', 'N_A6', 'freq_A7', 'N_A7', 'freq_AB8', 'N_AB8', 'freq_B1', 'N_B1', 'freq_B2', 'N_B2', 'freq_B3', 'N_B3', 'freq_B4', 'N_B4', 'freq_B5', 'N_B5', 'freq_B6', 'N_B6', 'freq_B7', 'N_B7']


In [99]:
print(list(snptable_X.T.columns[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]]))

['freq_A1', 'freq_A2', 'freq_A3', 'freq_A4', 'freq_A5', 'freq_A6', 'freq_A7', 'freq_AB8', 'freq_B1', 'freq_B2', 'freq_B3', 'freq_B4', 'freq_B5', 'freq_B6', 'freq_B7']


***Deleting all non 0/1 data and select only one of the 2 genotypes for each founder*** : loss of 45% of the SNPs.

In [100]:
snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns][:10]

,freq_A1,freq_A2,freq_A3,freq_A4,freq_A5,freq_A6,freq_A7,freq_AB8,freq_B1,freq_B2,freq_B3,freq_B4,freq_B5,freq_B6,freq_B7
0,0,0,0,0.99,0,0.91,0.91,0.32,0,0.85,0.04,0,0,0.12,0.93
1,0,0,0,0.98,0,0.86,0.91,0.32,0,0.79,0.02,0,0,0.11,0.89
2,1,1,1,1,1,1,1,1,0,1,1,0.77,1,1,1
3,0.06,0.04,0,0.99,0.12,0.83,0.93,0.71,0,0.83,0,0.1,0,0.08,0.8
4,0,0.04,0,0.99,0.18,0.9,0.96,0.86,0.17,0.8,0,0.06,0,0,0.84
5,0,0.12,0,0.99,0.17,0.86,0.98,0.86,0,0.8,0.02,0.04,0,0,0.91
6,0,0,0,0.99,0.05,0.89,0.81,0.84,0,0.82,0,0,0,0,0.97
7,0,0,0,0.99,0.04,0.88,0.8,0.84,0,0.81,0,0,0,0,0.97
8,0.51,0.76,0,0.97,0.78,0.84,0.92,0.73,0.53,0.73,0.53,0.64,0.02,0.49,0.83
9,0.04,0,0,0.99,0.37,0.78,0.85,0.68,0.39,0.73,0.01,0,0,0.04,0.61


In [129]:
snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns][np.logical_or(snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns]==0,snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns]==1)][:10]

,freq_A1,freq_A2,freq_A3,freq_A4,freq_A5,freq_A6,freq_A7,freq_AB8,freq_B1,freq_B2,freq_B3,freq_B4,freq_B5,freq_B6,freq_B7
0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN
1,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN
2,1,1,1,1,1,1,1,1,0,1,1,NaN,1,1,1
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,NaN
4,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN
5,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,NaN
6,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,NaN
7,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,NaN
8,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN


In [133]:
snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns][np.logical_or(snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns]==0,snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns]==1)].dropna(how='any')[:10]

,freq_A1,freq_A2,freq_A3,freq_A4,freq_A5,freq_A6,freq_A7,freq_AB8,freq_B1,freq_B2,freq_B3,freq_B4,freq_B5,freq_B6,freq_B7
24,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
25,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
27,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1
29,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
31,0,0,0,1,1,0,1,1,1,0,1,1,0,1,1
32,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
33,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
36,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
37,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1
39,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1


In [134]:
founders = snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns][np.logical_or(snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns]==0,snptable_X[4:].T[snptable_X[4:][[name[0]=="f" for name in snptable_X.T.columns[4:]]].T.columns]==1)].dropna(how='any')
founders[:10]

,freq_A1,freq_A2,freq_A3,freq_A4,freq_A5,freq_A6,freq_A7,freq_AB8,freq_B1,freq_B2,freq_B3,freq_B4,freq_B5,freq_B6,freq_B7
24,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
25,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
27,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1
29,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
31,0,0,0,1,1,0,1,1,1,0,1,1,0,1,1
32,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
33,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
36,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
37,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1
39,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1


In [136]:
founders.T.values.astype("int")[0,:10]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [137]:
F_geno = np.around(founders.T.values.astype("float"))
F_geno.shape

(15, 107252)

## Calculate similarity

In [138]:
def normalized_cross_similarity (array,n,p) :
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return max(0,np.round(np.mean(B),5)*2-1)    # Normalize between 0 and 1

In [139]:
def similarity_standard_deviation(array,n,p):
    A = np.dot(array,array.T)+np.dot(1-array,1-array.T)
    B = A[np.triu_indices(n, k=1)]/p     # Extract the values that are above the diagonal
    return np.round(np.sqrt(np.var(B)),3)

In [140]:
n, p = F_geno.shape

In [141]:
normalized_cross_similarity (F_geno,n,p)

0.4404999999999999

In [142]:
similarity_standard_deviation(F_geno,n,p)

0.024